In [42]:
import numpy as np
import pandas as pd
from collections import Counter

class Node:
    
    def __init__(self, root=True, label=None, feature_name=None, feature=None):
        self.root = root # true: 叶子节点 false:根结点或内部节点
        self.label = label
        self.feature_name = feature_name
        self.feature = feature
        self.tree = {}
        self.result = {
            'label' : self.label,
            'feature' : self.feature,
            "tree" : self.tree
        }
        
    def __repr__(self):
        return '{}'.format(self.result)
    
    def add_node(self, val, node):
        self.tree[val] = node
        
    def predict(self, features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature]].predict(features)

class CARTTree:
    
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon
        self._tree = {}
        
    # 计算传入数据的Gini指数
    def calcGini(self,dataSet):
        # 获得y中分类标签的唯一值
        y_lables = np.unique(dataSet[:,-1])
        y_counts=len(dataSet) # y总数据条数
        y_p={}             # y中每一个分类的概率，字典初始化为空，y分类数是不定的，按字典存储更方便取值
        gini=1.0
        for y_lable in y_lables:
            y_p[y_lable]=len(dataSet[dataSet[:, -1]==y_lable])/y_counts  # y中每一个分类的概率（其实就是频率）
            gini-=y_p[y_lable]**2
        return gini

    #划分数据集
    def splitDataSet(self,dataSet, i, value,types=1):
        #dataSet[list(dataSet[:,0]!='青年')]
        if types==1: # 使用此列特征中的value进行划分数据
            subDataSet=dataSet[list(dataSet[:,i]==value)]  # 按照数据x第i列==value即可判断，不需要像《机器学习实战》书里写的那么复杂
            subDataSet = np.array(subDataSet)           # 强制转换为array类型
        elif types==2: # 使用此列特征中的不等于value的进行划分数据
            subDataSet=dataSet[list(dataSet[:,i]!=value)]  # 按照数据x第i列==value即可判断，不需要像《机器学习实战》书里写的那么复杂
            subDataSet = np.array(subDataSet)           # 强制转换为array类型
        return subDataSet,len(subDataSet)


    # 计算Gini指数，选择最好的特征划分数据集，即返回最佳特征下标及传入数据集各列的Gini指数
    def chooseBestFeature(self,dataSet):
        numTotal=dataSet.shape[0]              # 记录本数据集总条数
        numFeatures = len(dataSet[0]) - 1      # 最后一列为y，计算x特征列数
        bestFeature = -1                       # 初始化参数，记录最优特征列i，下标从0开始
        columnFeaGini={}                       # 初始化参数，记录每一列x的每一种特征的基尼 Gini(D,A)
        for i in range(numFeatures):           # 遍历所有x特征列
            # i=2
            prob = {}                          # 按x列计算各个分类的概率
            featList = list(dataSet[:,i])      # 取这一列x中所有数据，转换为list类型
            prob=dict(Counter(featList))       # 使用Counter函数计算这一列x各特征数量
            for value in prob.keys():          # 循环这一列的特征，计算H(D|A)
                # value='是'
                feaGini = 0.0
                bestFlag = 1.00001  # 对某一列x中，会出现x=是，y=是的特殊情况，这种情况下按“是”、“否”切分数据得到的Gini都一样，设置此参数将所有特征都乘以一个比1大一点点的值，但按某特征划分Gini为0时，设置为1
                subDataSet1,sublen1 = self.splitDataSet(dataSet, i, value, 1)  # 获取切分后的数据
                subDataSet2,sublen2 = self.splitDataSet(dataSet, i, value, 2)
                if (sublen1/numTotal) * self.calcGini(subDataSet1)==0: # 判断按此特征划分Gini值是否为0（全部为一类）
                    bestFlag = 1 
                feaGini += (sublen1/numTotal) * self.calcGini(subDataSet1) + (sublen2/numTotal) * self.calcGini(subDataSet2)
                columnFeaGini['%d_%s'%(i,value)]=feaGini*bestFlag
        bestFeature=min(columnFeaGini,key=columnFeaGini.get) # 找到最小的Gini指数益对应的数据列
        bestFeatureIndex = int(bestFeature.split('_')[0])
        return bestFeatureIndex,columnFeaGini[bestFeature]
        
    def train(self,train_data):
        y_train,feature_names = train_data.iloc[:,-1],train_data.columns[:-1]
        if len(y_train.value_counts()) == 1:
            return Node(root=True,label=y_train.iloc[0])             
        if len(feature_names)==0:
            return Node(root=True,label=y_train.value_counts().sort_values(ascending=False).index[0])
     
        max_feature,max_info_gain_ratio = self.chooseBestFeature(np.array(train_data))
        
        max_feature_name = feature_names[int(max_feature)]
        print("maxFeature:%s"%max_feature)
        print("maxFeature:%s"%max_feature_name)
            
        if max_info_gain_ratio < self.epsilon:
            return Node(
                    root=True,
                    label=y_train.value_counts().sort_values(
                    ascending=False).index[0])
        
        node_tree = Node(root = False,feature_name=max_feature_name,feature = max_feature)
        
        feature_list = train_data[max_feature_name].value_counts().index
        
        for f in feature_list:
            sub_train_df = train_data.loc[train_data[max_feature_name] ==
                                          f].drop([max_feature_name], axis=1)
            #递归生成树
            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f, sub_tree)
            
        return node_tree
                            
    def fit(self, train_data):
        self._tree = self.train(train_data)
        return self._tree

    def predict(self, X_test):
        return self._tree.predict(X_test)

datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
train_data = pd.DataFrame(datasets, columns=labels)
dt = CARTTree()


tree = dt.fit(train_data)
tree


maxFeature:2
maxFeature:有自己的房子
maxFeature:1
maxFeature:有工作


{'label': None, 'feature': 2, 'tree': {'否': {'label': '否', 'feature': None, 'tree': {}}, '是': {'label': '是', 'feature': None, 'tree': {}}}}

In [44]:
dt.predict(['青年', '是', '是', '一般'])

'是'